# Restmateriaal

We onderzoeken hier verschillende mogelijkheden voor het maken van een dergelijke gateway:

* computer (bijv. Raspberry Pi) via USB verbonden met een microbit
* microbit met WiFi radio (elecfreaks iot-bit

De Elecfreaks iot-bit wordt gewoonlijk gebruikt om een enkele microbit te verbinden met het internet. Maar we kunnen deze ook programmeren als gateway, om een lokaal microbit-netwerk te verbinden met het internet.

Voor het gebruik van een gateway in een *schoolnetwerk* moet deze aan een aantal extra eisen voldoen:

* geen gebruik van andere poorten dan 80 en 433 (http, https); veel andere poorten en protocollen zijn vaak geblokkeerd;
* voor een WiFi verbinding: de mogelijkheid om te verbinden met een netwerk met een wachtwoord per computer (of per gebruiker), in plaats van een wachtwoord voor het hele netwerk.
* (een alternatief is het gebruik van een bedraad netwerk; dan heb je niet het probleem dat je moet inloggen op het netwerk.

Voor het verbinden van de IoT-gateway met het internet gebruiken we meestal het MQTT protocol; dit biedt symmetrische "publish-subscribe" communicatie mogelijk, in tegenstelling tot de client-server communicatie van HTTP.
Het MQTT protocol gebruikt gewoonlijk een TCP-verbinding via de poorten 1883/1884. Maar het is tegenwoordig ook mogelijk om een "websockets" (HTTP) verbinding te gebruiken als transportlaag voor MQTT. Daarmee omzeil je de poort-beperkingen van een schoolnetwerk.

---

## Uit het overzicht.

### Bijeenkomst 1

* wat is het internet of things? - als ontwikkeling van het internet, na het web.
* waarom het internet of things? - slimme oplossingen, betere beslissingen
* hoe het internet of things? - onderdelen en hun verbindingen, relaties
    * eerst als functionele onderdelen ("eindapparaten" vanuit het netwerk gezien)
    * dan met verbindingen en verbindingsapparaten
* opbouw van een IoT-knoop - 
* protocollen: MQTT;
* MQTT - publish/subscribe principe; topic, payload keuzes (JSON)
* draadloze IoT-knopen: keuze van een radio
    * hier: microbit-radio (met eigenschappen)

Praktische opdrachten:

* MQTT - publish/subscribe (oefenen met MQT3)
* NodeRed - 
* microbit IoT-knoop, microbit (pakket) radio
    * programmeren van de microbit IoT-knoop (te volgen via host)
    * programmeren van de logger-knoop (als beschikbaar
    * LPP formaat; en (via logger) JSON formaat
    * volgende keer: verbinden aan het internet (MQTT)

### Bijeenkomst 2

* gateway: functie, opbouw; protocollen
* MQTT: adressering van knopen, (sensoren?); netwerk?; 

Praktische opdrachten:

* programmeren (installeren) van de gateway; experimenteren met het lokale netwerk en MQTT
* NodeRed: monitoring van het IoT-netwerk
* NodeRed: MQTT verbindingen; publish, subscribe in NodeRed
* NodeRed: interpreteren van JSON-berichten; dashboard

### Bijeenkomst 3

* (automatisch) sturen: controllers
    * NodeRed als voorbeeld
* koppelen van sensoren en actuatoren; sturen

Praktische opdrachten:

* NodeRed: (automatisch) sturen (condities in flows)

### Open punten

* user interface, dashboard
* opslag van sensor-gegevens
    * per knoop? of per sensor? (of allebeide?)
* identificatie van sensoren - (en van knopen); op het niveau van de toepassing
* 

### Opdrachten

Eerste bijeenkomst:

* kennismaking MQTT (vooral via web-app)
* kennismaking JSON (idem; )
* kennismaking NodeRed (met JSON, MQTT berichten)
* kennismaking LPP (binaire codering)

| opdracht | omschrijving |
|:--       | :---         |
| **mqtt**     |  |
| x.x      | chat-0: mqtt publish/subscribe via web-app |
| x.x      | chat-1: mqtt publish/subscribe, met topic en wildcard |
| x.x.     | sensors en actuators: mqtt publish/subscribe, via web-app |
| **nodered** | |
| b.b      | ontvangen en verwerken van elementaire berichten, koppelen |
| b.b      | ontvangen van mqtt-berichten (nb: in JSON) |
| **lpp**         |              |
| a.a      | lpp gecodeerde berichten via pakketradio (radio logger) |
| a.a      | lpp omgezet naar json (radio logger) |

NodeRed is een *controller* waarmee je de verschillende onderdelen (en protocollen) van de IoT-keten aan elkaar kunt koppelen.

Tweede bijeenkomst:

* nadruk op de gateway,
* verwerken van de sensordata (dashboard?)
* omzetten van JSON in LPP (v.v.)

| opdracht | omschrijving |
|:--       | :---         |
| **mqtt**     |  |
| x.x      | chat-0: mqtt publish/subscribe via web-app |
| x.x      | chat-1: mqtt publish/subscribe, met topic en wildcard |
| x.x.     | sensors en actuators: mqtt publish/subscribe, via web-app |
| **nodered** | |
| b.b      | ontvangen en verwerken van elementaire berichten, koppelen |
| b.b      | ontvangen van mqtt-berichten (nb: in JSON) |
| **lpp**         |              |
| a.a      | lpp gecodeerde berichten via pakketradio (radio logger) |
| a.a      | lpp omgezet naar json (radio logger) |

Derde bijeenkomst:

* sturen met NodeRed
* 

| opdracht | omschrijving |
|:--       | :---         |
| **mqtt**     |  |
| x.x      | chat-0: mqtt publish/subscribe via web-app |
| x.x      | chat-1: mqtt publish/subscribe, met topic en wildcard |
| x.x.     | sensors en actuators: mqtt publish/subscribe, via web-app |
| **nodered** | |
| b.b      | ontvangen en verwerken van elementaire berichten, koppelen |
| b.b      | ontvangen van mqtt-berichten (nb: in JSON) |
| **lpp**         |              |
| a.a      | lpp gecodeerde berichten via pakketradio (radio logger) |
| a.a      | lpp omgezet naar json (radio logger) |